In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#application of image pyramid is image blending and reconstruction of image
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')
print(apple.shape)
print(orange.shape)
#first create the halve of these two
apple_orange = np.hstack((apple[:,:256],orange[:,256:])) #doing this will merge the image but there will be visible
#seperation line. To blend it perfectly, we need to follow 5 steps
# 1. Load the two images
# 2. Find the gaussian pyramid for both with level 6
# 3. from gaussian find their laplacian
# 4. Join the left half of apple and right half of orange in each level
# 5. From this joint image pyramid, reconstruct them

cv2.imshow('Apple',apple)
cv2.imshow('Orange',orange)
cv2.imshow('Apple_Orange',apple_orange)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512, 3)
(512, 512, 3)


### Image blending

In [28]:
#generate gaussian for apple
apple_copy = apple.copy()
gp_apple = [apple_copy]

for i in range(6):
    layer = cv2.pyrDown(apple_copy)
    gp_apple.append(layer)
print(len(gp_apple))

#generate gaussian for apple
orange_copy = orange.copy()
gp_orange = [orange_copy]

for i in range(6):
    layer = cv2.pyrDown(orange_copy)
    gp_orange.append(layer)
print(len(gp_orange))

#generate laplacian for apple
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5,0,-1):
    gaussian_extended = cv2.pyrUp(gp_apple[i])
    laplacian = cv2.subtract(gp_apple[i-1],gaussian_extended)
    lp_apple.append(laplacian)
print(len(lp_apple))
    
#generate laplacian for orange
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5,0,-1):
    gaussian_extended = cv2.pyrUp(gp_orange[i])
    laplacian = cv2.subtract(gp_orange[i-1],gaussian_extended)
    lp_orange.append(laplacian)
print(len(lp_orange))


#now add left and right halves of images
apple_orange_pyramid = []
n = 0

for apple_lap,orange_lap in zip(lp_apple,lp_orange):
    n += 1
    cols, rows, channels = apple_lap.shape
    laplacian = np.hstack((apple_lap[:,0:int(cols/2)]),(orange_lap[:,int(cols/2):]))
    apple_orange_pyramid.append(laplacian)
    
#reconstruct the image
ap_or_reconstruct = [apple_orange_pyramid]
for i in range(1,6):
    ap_or_reconstruct = cv2.pyrUp(ap_or_reconstruct)
    ap_or_reconstruct = cv2.add(apple_orange_pyramid[i],ap_or_reconstruct)

cv2.imshow('APple Oragne', ap_or_reconstruct)
cv2.waitKey(0)
cv2.destroyAllWindows()

7
7


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\core\src\arithm.cpp:669: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


In [49]:
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')
#print(apple.shape)
#print(orange.shape)
cv2.imshow('Image',apple)

apple_copy = apple.copy()
print(apple_copy.shape)
for i in range(4):
    apple_copy = cv2.pyrDown(apple_copy)
    print(apple_copy.shape)
    cv2.imshow(str(i),apple_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512, 3)
(256, 256, 3)
(128, 128, 3)
(64, 64, 3)
(32, 32, 3)


In [10]:
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')

#generate gaussian for apple
apple_copy = apple.copy()
first = [apple_copy]
for i in range(4):
    apple_copy = cv2.pyrDown(apple_copy)
    first.append(apple_copy)
    
for i in range(5):
    print(first[i].shape)
print('\n')
 
#generate gaussian for orange
orange_copy = orange.copy()
second = [orange_copy]
for i in range(4):
    orange_copy = cv2.pyrDown(orange_copy)
    second.append(orange_copy)
    
#Laplacian for apple
lpF = [first[4]]
print('Lap First: ',lpF[0].shape)

for i in range(4,0,-1):
    size = (first[i - 1].shape[1],first[i - 1].shape[0])
    first_expanded = cv2.pyrUp(first[i],dstsize=size)
    laplacian = cv2.subtract(first[i-1], first_expanded)
    lpF.append(laplacian)

#Laplacian for orange
lpS = [second[4]]
print('Lap Second: ',lpS[0].shape)

for i in range(4,0,-1):
    size = (second[i - 1].shape[1],second[i - 1].shape[0])
    second_expanded = cv2.pyrUp(second[i],dstsize=size)
    laplacian = cv2.subtract(second[i-1], second_expanded)
    lpS.append(laplacian) 

LS = [] #empty list for reconstruction Image
n = 0
for la,lb in zip(lpF,lpS):
    cols,rows,ch = la.shape
    n += 1
    ls = np.hstack((la[:,:int(cols/2)],lb[:,int(cols/2):]))
    LS.append(ls)
print('Length of LS: ',len(LS))
reconstructed_image = LS[0]

for i in range(1,5):
    size = (LS[i].shape[1], LS[i].shape[0])
    reconstructed_image = cv2.pyrUp(reconstructed_image, dstsize=size)
    reconstructed_image = cv2.add(LS[i],reconstructed_image)
cv2.imshow("blended image",reconstructed_image)   # showing new reconstructed image
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512, 3)
(256, 256, 3)
(128, 128, 3)
(64, 64, 3)
(32, 32, 3)


Lap First:  (32, 32, 3)
Lap Second:  (32, 32, 3)
Length of LS:  5


###  Self Practice

In [4]:
apple = cv2.imread('apple.jpg')
orange = cv2.imread('orange.jpg')

blend = np.hstack((apple[:,:256], orange[:,256:]))

cv2.imshow('Blend', blend)

print(apple.shape)
print(orange.shape)

cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512, 3)
(512, 512, 3)


In [11]:
# Image Blending
#generate apple gaussian
apple = cv2.imread('apple.jpg')
gp_apple = [apple]

for i in range(6):
    layer = cv2.pyrDown(apple)
    apple = layer
    gp_apple.append(apple)
    
#generate orange gaussian
orange = cv2.imread('orange.jpg')
gp_orange = [orange]

for i in range(6):
    layer = cv2.pyrDown(orange)
    orange = layer
    gp_orange.append(orange)
    
#Generate Laplacian for apple
last_apple = gp_apple[-1]
lap_apple = [last_apple]

for i in range(6,0,-1):
    gp_apple_extend = cv2.pyrUp(gp_apple[i])
    Laplacian = cv2.subtract(gp_apple[i-1],gp_apple_extend)
    lap_apple.append(Laplacian)
    
#Generate Laplacian for orange
last_orange = gp_orange[-1]
lap_orange = [last_orange]

for i in range(6,0,-1):
    gp_orange_extend = cv2.pyrUp(gp_orange[i])
    Laplacian = cv2.subtract(gp_orange[i-1],gp_orange_extend)
    lap_orange.append(Laplacian)

#Now add two images
apple_orange_pyr = []
n = 0
for lp_apple,lp_orange in zip(lap_apple,lap_orange):
    n += 1
    cols,rows,channels = lp_apple.shape
    Laplacian_stack = np.hstack((lp_apple[:,:int(cols/2)], lp_orange[:,int(cols/2):]))
    apple_orange_pyr.append(Laplacian_stack)

#reconstruct the image
app_or_reconstruct = apple_orange_pyr[0]
for i in range(1,7):
    size = (apple_orange_pyr[i].shape[1], apple_orange_pyr[i].shape[0])
    app_or_reconstruct = cv2.pyrUp(app_or_reconstruct, dstsize=size)
    app_or_reconstruct = cv2.add(app_or_reconstruct,apple_orange_pyr[i])
    
cv2.imshow('Blended Image', app_or_reconstruct)
cv2.waitKey(0)
cv2.destroyAllWindows()
    
